In [2]:
from underthesea import pos_tag
import re
import string
import csv
import pandas as pd
import numpy as np

In [12]:
def prepros():
    dic = {}
            
    for fi in ['train.txt']:
        with open(fi, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            cnt = -1
            for para in lines:
                cnt += 1
                category, para = para.split('\t', 1)
                for s in pos_tag(para):
                    if s[1] in ('A', 'V') or re.search('^N', s[1]):
                        w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
                        if w != '':
                            if w not in dic:
                                dic[w] = [0] * len(lines)
                            dic[w][cnt] += 1

    df = pd.DataFrame(dic)
    df.to_csv('featureVectors_5kS.csv')
    
    with open('test.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for k in dic:
            dic[k] = [0] * len(lines)
        cnt = -1
        for para in lines:
            cnt += 1
            for s in pos_tag(para):
                if s[1] in ('A', 'V') or re.search('^N', s[1]):
                    w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
                    if w != '':
                        if w not in dic:
                            continue
                        dic[w][cnt] += 1
    df = pd.DataFrame(dic)
    df.to_csv('testVectors.csv')

In [8]:
prepros()

,Unnamed: 0,gần,học sinh,phải,học,điều kiện,tạm bợ,nlđ,trường,tiểu học,...,alec,baldwin,eminem,meirelles,đại náo,hector babenco,giải nam,william hurt,city of god,maids
0,0,3,2,3,5,1,2,1,4,2,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473,3473,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3474,3474,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3475,3475,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3476,3476,0,0,1,0,0,0,0,0,0,...,1,3,1,0,0,0,0,0,0,0


In [25]:
%%time
df = pd.read_csv('featureVectors_5kS.csv')
x_train = df.iloc[:, 1:].values
y_train, y_test = [], []
with open('train.txt', 'r', encoding='utf-8') as f:
    for para in f.readlines():
        y_train.append(para.split('\t')[0])
        
df1 = pd.read_csv('testVectors.csv')
x_test = df1.iloc[:, 1:].values
with open('testAns.txt', 'r', encoding='utf-8') as f:
    for para in f.readlines():
        y_test.append(para.strip())

Wall time: 2min 2s


In [4]:
with open('scrapy_test/spiders/Output/_trainSet.txt', 'r', encoding='utf-8') as f:
    for para in f.readlines():
        y_train.append(para.split('\t')[0])
x_test = np.concatenate([np.array(x_test), np.zeros((1522, 57617-46021))], axis=1)

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

y_pred = MultinomialNB().fit(x_train, y_train).predict(x_test)
accuracy_score(y_true=y_test, y_pred=y_pred) * 100
for i in range(len(y_test)):
    if y_test[i] != y_pred[i]:
        print(i, ' : ', 'ans: ', y_test[i], '  pred: ', y_pred[i])

8  :  ans:  __CTXH__   pred:  __PL__
10  :  ans:  __CTXH__   pred:  __SK__
12  :  ans:  __CTXH__   pred:  __PL__
16  :  ans:  __CTXH__   pred:  __DS__
21  :  ans:  __CTXH__   pred:  __DS__
25  :  ans:  __CTXH__   pred:  __KD__
28  :  ans:  __CTXH__   pred:  __DS__
33  :  ans:  __CTXH__   pred:  __PL__
47  :  ans:  __CTXH__   pred:  __VH__
48  :  ans:  __CTXH__   pred:  __KH__
50  :  ans:  __CTXH__   pred:  __DS__
56  :  ans:  __CTXH__   pred:  __KD__
58  :  ans:  __CTXH__   pred:  __DS__
64  :  ans:  __CTXH__   pred:  __DS__
67  :  ans:  __CTXH__   pred:  __KD__
69  :  ans:  __CTXH__   pred:  __PL__
72  :  ans:  __CTXH__   pred:  __DS__
76  :  ans:  __CTXH__   pred:  __KD__
80  :  ans:  __CTXH__   pred:  __PL__
81  :  ans:  __CTXH__   pred:  __KD__
85  :  ans:  __CTXH__   pred:  __PL__
87  :  ans:  __CTXH__   pred:  __DS__
88  :  ans:  __CTXH__   pred:  __KD__
94  :  ans:  __CTXH__   pred:  __PL__
96  :  ans:  __CTXH__   pred:  __DS__
98  :  ans:  __CTXH__   pred:  __PL__
99  :  ans:  

In [20]:
from sklearn.metrics import accuracy_score
from sklearn import tree

mytree = tree.DecisionTreeClassifier(max_depth=500, min_samples_split=30, min_impurity_decrease=0.001)
y_pred = mytree.fit(x_train, y_train).predict(x_test)
print(accuracy_score(y_true=y_test, y_pred=y_pred) * 100)
#tree.plot_tree(mytree)

65.17739816031536


In [3]:
dic = {}
df = pd.read_csv('featureVectors_5kS.csv').set_index('Unnamed: 0')
df

,Unnamed: 0,gần,học sinh,phải,học,điều kiện,tạm bợ,nlđ,trường,tiểu học,...,alec,baldwin,eminem,meirelles,đại náo,hector babenco,giải nam,william hurt,city of god,maids
0,0,3,2,3,5,1,2,1,4,2,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473,3473,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3474,3474,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3475,3475,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3476,3476,0,0,1,0,0,0,0,0,0,...,1,3,1,0,0,0,0,0,0,0


In [7]:
fi = open('scrapy_test/spiders/Output/_trainSet.txt', 'r', encoding='utf-8')
paras = fi.readlines()
for w in df.columns:
    dic[w] = [0] * 1106
cnt = -1
for para in paras:
    cnt += 1
    category, para = para.split('\t', 1)
    for s in pos_tag(para):
        if s[1] in ('A', 'V') or re.search('^N', s[1]):
            w = re.sub(r'[' + string.punctuation + ']', ' ', s[0]).strip().lower()
            if w != '':
                if w not in dic:
                    dic[w] = [0] * 1106
                dic[w][cnt] += 1

df3 = pd.DataFrame(dic)
df3

,gần,học sinh,phải,học,điều kiện,tạm bợ,nlđ,trường,tiểu học,đống đa,...,pvcombank 49 4,đông đô,đặng thị quỳnh hương,quản trọng đức,jeongho,landmark,kiểm soát viên,bùi văn tuấn,phạm thị ngọc lan,phậm
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,2,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,2,2,0,0,0,8,5,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1104,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df4 = pd.concat([df, df3], axis=0).fillna(0).astype(int)
df4

,gần,học sinh,phải,học,điều kiện,tạm bợ,nlđ,trường,tiểu học,đống đa,...,pvcombank 49 4,đông đô,đặng thị quỳnh hương,quản trọng đức,jeongho,landmark,kiểm soát viên,bùi văn tuấn,phạm thị ngọc lan,phậm
0,3,2,3,5,1,2,1,4,2,2,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1104,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df4.to_csv('featureVectors_6kS.csv')